In [1]:
# Load all necessary Packages
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras import initializers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# supporting libraries
import numpy as np
import time
import os
import copy
import pickle
import h5py

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [25]:
# Define constants
IMAGE_SIZE =(32, 32)  # Assuming input images are grayscale # Size to which images will be resized
NUM_CLASSES = 12  # Number of classes (subfolders)

In [26]:
# Function to preprocess and augment images
def preprocess_and_augment(image):
    # Resize image
    resized_image = cv2.resize(image, IMAGE_SIZE)
    # Convert to grayscale
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values
    normalized_image = grayscale_image / 255.0
    return normalized_image


In [27]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Degree range for random rotations
    width_shift_range=0.1,  # Fraction of total width for horizontal shifts
    height_shift_range=0.1,  # Fraction of total height for vertical shifts
    shear_range=0.2,  # Shear angle in counter-clockwise direction
    zoom_range=0.2,  # Range for random zoom
    horizontal_flip=True,  # Randomly flip inputs horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)


In [28]:
# Load images from folders
def load_data(data_dir):
    images = []
    labels = []
    for class_index in range(NUM_CLASSES):
        class_dir = os.path.join(data_dir, str(class_index))
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = cv2.imread(image_path)
            preprocessed_image = preprocess_and_augment(image)
            images.append(preprocessed_image)
            labels.append(class_index)
    return np.array(images), np.array(labels)

In [29]:
# Load training data
train_data_dir = "/content/drive/MyDrive/HANDWRITTENTAMILCHARACTERS/processed/train"  # Replace with your train folder path
X_train, y_train = load_data(train_data_dir)


In [30]:
# Shuffle the data
shuffle_indices = np.random.permutation(len(X_train))
X_train = X_train[shuffle_indices]
y_train = y_train[shuffle_indices]

In [31]:
def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    return model

In [32]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)


In [33]:
input_shape

(32, 32, 1)

In [34]:
model = create_cnn_model(input_shape)


In [35]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)             

In [36]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
102/102 [==============================] - 5s 34ms/step - loss: 1.9433 - accuracy: 0.3236 - val_loss: 1.2807 - val_accuracy: 0.5670
Epoch 2/10
102/102 [==============================] - 4s 43ms/step - loss: 0.9873 - accuracy: 0.6850 - val_loss: 0.7948 - val_accuracy: 0.7491
Epoch 3/10
102/102 [==============================] - 4s 37ms/step - loss: 0.6699 - accuracy: 0.7831 - val_loss: 0.7477 - val_accuracy: 0.7638
Epoch 4/10
102/102 [==============================] - 3s 32ms/step - loss: 0.5019 - accuracy: 0.8321 - val_loss: 0.5197 - val_accuracy: 0.8266
Epoch 5/10
102/102 [==============================] - 3s 32ms/step - loss: 0.3714 - accuracy: 0.8770 - val_loss: 0.5002 - val_accuracy: 0.8278
Epoch 6/10
102/102 [==============================] - 5s 47ms/step - loss: 0.2995 - accuracy: 0.9000 - val_loss: 0.3408 - val_accuracy: 0.8881
Epoch 7/10
102/102 [==============================] - 3s 32ms/step - loss: 0.2437 - accuracy: 0.9194 - val_loss: 0.3208 - val_accuracy: 0.8942

In [38]:
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
    resized_image = cv2.resize(image, (32, 32))  # Resize image to (32, 32)
    normalized_image = resized_image / 255.0  # Normalize pixel values
    return np.expand_dims(normalized_image, axis=-1)  # Add channel dimension

In [68]:
# Load an image for prediction
image_path = '/content/drive/MyDrive/HANDWRITTENTAMILCHARACTERS/processed/test/7/01720.tiff'  # Replace with the path to your image
input_image = preprocess_image(image_path)

In [69]:
# Make prediction
prediction = model.predict(np.array([input_image]))
predicted_class_index = np.argmax(prediction)


1/1 [==============================] - 0s 33ms/step


In [70]:
# Print the predicted class index
print("Predicted class index:", predicted_class_index)

Predicted class index: 7


In [ ]:
# Function to preprocess and resize a single image (TIFF)
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
    if image is None:
        raise FileNotFoundError(f"Could not read image file: {image_path}")
    resized_image = cv2.resize(image, (32, 32))  # Resize image to (32, 32)
    normalized_image = resized_image / 255.0  # Normalize pixel values
    return np.expand_dims(normalized_image, axis=-1)  # Add channel dimension


In [45]:
# Save the model
model.save("tamil_handwritten_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [46]:
from google.colab import files

# Path to the model file in Google Colab
colab_model_path = 'tamil_handwritten_model.h5'

# Download the file
files.download(colab_model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>